In [1]:
import numpy as np
from src.jss_lite.jss_lite import jss_lite
t=np.empty((4,1),dtype=object)
import time
import os
env=jss_lite(instance_path='resources/jsp_instances/standard/abz5.txt')
state=env.get_state()
env.set_state(state)

OrderedDict([('action_mask',
              array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
                     0., 0., 0.])),
             ('obs',
              array([1.    , 0.    , 0.8889, 0.    , 0.    , 0.    , 1.    , 0.    ,
                     0.7273, 0.    , 0.    , 0.    , 1.    , 0.    , 0.8384, 0.    ,
                     0.    , 0.    , 1.    , 0.    , 0.9495, 0.    , 0.    , 0.    ,
                     1.    , 0.    , 0.697 , 0.    , 0.    , 0.    , 1.    , 0.    ,
                     1.    , 0.    , 0.    , 0.    , 1.    , 0.    , 0.5051, 0.    ,
                     0.    , 0.    , 1.    , 0.    , 0.9899, 0.    , 0.    , 0.    ,
                     1.    , 0.    , 0.9495, 0.    , 0.    , 0.    , 1.    , 0.    ,
                     0.5051, 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
                     0.    , 0.    , 1.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
                     1.    , 0.    , 0.    , 0.    , 0.    ,

In [6]:
import numpy as np
from src.jss_lite.jss_lite import jss_lite
t=np.empty((4,1),dtype=object)
import time
import os
from random import randrange
env=jss_lite(instance_path='resources/jsp_instances/standard/abz5.txt')
state=env.get_state()
env.set_state(state)
token=False
for _ in range(99):
    state=env.reset()
    done=False
    i=0
    #print(state['obs'].shape)
    #for i in range(150):
    while not done:
        i+=1
        legal_action=env.get_legal_actions(state)
        #action=np.random.choice(len(legal_action), 1, p=(legal_action / legal_action.sum()))[0]
        action=randrange(env.action_space.n)
        #print(action)
        if action > env.action_space.n/2:
            token=True
            #print("before:")
            #env.render(start_count=1,x_bar="Job",y_bar="Machine")
            state, reward, done, info=env.step(action)
            #print(state['obs'].shape)

        else:
            state, reward, done, info=env.step(action)  
            #print(state['obs'].shape)
            if token:
                token=False
                #print("after:")
                #env.render(start_count=1,x_bar="Job",y_bar="Machine")
        #print(action)
        #tmp=(env.processed_and_max_time_job_matrix)
        #print(env.count_finished_tasks_machine_matrix)
        #print(env.production_list)
        #env.render()
        if done==True:
            print(f"finished! no legal actions possible after {i+1} steps")
            #print(env.timestemp_list)
            break
    #env.render(start_count=1,x_bar="Job",y_bar="Machine")



(120,)
19
16
18
9
14
16
7
8
14
6
9
9
3
17
15
3
7
17
11
13
10
12
8
8
14
9
7
16
16
5
11
8
15
14
19
13
12
16
9
1
3
4
12
9
11
17
0
19
3
18
18
13
17
9
5
4
8
18
7
10
1
17
6
0
12
8
10
4
12
10
17
9
14
10
2
12
3
14
8
12
18
8
14
2
7
13
6
5
10
13
6
7
3
7
11
19
0
2
6
12
9
14
8
9
0
8
14
19
9
11
8
15
16
1
4
15
18
8
15
16
10
16
1
17
5
5
10
2
10
15
6
12
19
15
16
4
8
5
19
1
3
5
2
13
8
19
14
13
8
12
16
19
7
10
1
2
6
10
9
7
11
5
8
11
12
19
18
3
2
6
6
4
16
4
7
5
14
1
6
13
4
4
9
5
11
15
1
9
12
11
11
17
1
6
17
7
16
6
3
2
13
4
19
8
9
6
9
11
15
4
14
12
6
8
7
16
5
8
5
9
7
12
2
11
0
15
12
9
5
10
7
5
9
11
12
11
10
17
1
15
14
6
6
0
7
10
13
2
0
5
8
0
1
9
17
3
11
2
17
6
7
12
0
1
12
3
18
9
5
15
8
11
1
1
11
13
13
1
13
13
5
6
11
8
14
2
16
18
4
12
4
9
5
12
14
4
10
15
15
11
6
4
13
11
19
19
0
15
7
19
10
5
4
2
0
4
10
17
2
19
12
7
1
1
15
19
10
5
2
18
17
17
9
9
10
9
8
7
18
15
5
1
9
5
7
7
10
10
2
15
2
15
7
7
12
1
3
18
5
6
16
4
15
8
14
13
2
17
13
17
14
0
1
10
2
1
18
19
18
17
4
15
4
13
4
5
18
10
18
13
19
18
11
0
8
12
19
10
18


(120,)
18
18
3
4
5
14
0
13
10
17
16
17
6
16
19
2
13
3
1
12
17
3
17
11
15
17
0
6
5
7
12
12
4
13
1
10
12
19
1
4
15
11
6
2
2
4
5
15
11
6
10
12
17
11
4
18
10
17
15
18
10
15
6
3
9
11
8
5
2
6
17
3
10
19
1
16
4
1
9
19
16
11
18
16
7
19
7
5
2
11
17
18
0
1
14
17
14
2
1
0
5
19
2
0
8
10
16
6
9
15
15
5
14
13
6
11
6
0
4
4
0
3
7
12
8
15
0
3
8
1
14
16
6
1
2
9
4
14
19
3
5
5
17
7
17
6
1
16
1
19
4
1
15
11
14
5
14
12
0
15
1
16
17
1
10
5
10
16
19
6
5
2
12
15
13
3
16
16
10
12
8
2
11
5
7
8
0
16
11
11
1
7
9
4
7
4
16
2
19
19
2
5
3
2
19
10
7
15
15
12
1
1
17
4
8
16
13
12
4
16
3
1
0
13
9
14
19
9
10
16
16
12
18
4
1
14
17
4
11
16
9
7
12
0
16
16
1
13
2
9
4
5
0
7
6
11
10
14
2
4
16
0
3
15
1
7
1
3
1
5
5
14
11
1
19
15
6
13
2
4
11
14
3
11
10
6
8
18
4
10
19
5
12
11
10
6
17
6
13
19
13
19
4
9
15
11
2
9
17
10
7
13
9
15
13
9
18
11
14
4
6
13
3
8
19
0
1
6
10
4
14
0
6
6
16
8
0
5
13
5
16
1
7
5
6
15
17
13
0
14
1
7
18
18
13
14
2
12
5
8
6
2
8
5
4
10
11
0
5
18
2
2
1
10
18
9
11
18
11
18
11
15
0
19
0
0
19
6
5
19
2
10
18
0
3
5
11
14
2
2

KeyboardInterrupt: 

In [4]:
print(env.observation[0,0])


1.0


In [ ]:
print(env.processed_and_max_time_job_matrix)

In [ ]:
print(env.observation)

In [ ]:
import plotly.express as px
x_bar="Machine"
y_bar="Job"
df.sort_values(inplace=True,by="Job")
fig = px.timeline(df, x_start="Start", x_end="Finish", color=x_bar, y=y_bar)

fig.show()

In [ ]:
print(action)

print(env.get_legal_actions(state))
print(env.job_machine_matrix[action,env.count_finished_tasks_job_matrix[action]])
cur_machine=env.current_machines_status
job_ma=env.job_machine_matrix
print(env.count_finished_tasks_job_matrix[action])
#print(env.current_machines_status[env.job_machine_matrix[action,env.count_finished_tasks_job_matrix[action]],:])
#print(env.job_machine_matrix[action,env.count_finished_tasks_job_matrix[action]])
#print(env.current_machines_status)
env.render()


In [ ]:
import ale_py as ale
ale.importroms('/Users/felix/sciebo/masterarbeit/progra/model-based_rl/roms/Space Invaders.bin')
import gym
env = gym.make('SpaceInvaders-v4')
env.reset()
env.render()


In [ ]:
import JSSEnv
import src.jss_graph_env.disjunctive_graph_jss_env as jss_env
import src.jsp_instance_parser 
import time
import gym
import pandas as pd
import random
import numpy as np
from src.jss_lite.jss_lite import jss_lite

index=['klagenfurt','alex','lite']
df=pd.DataFrame(np.zeros((3,5)),columns=['setuptime','resettime','50xactiontime','col4','col5'],index=index)
#df=pd.DataFrame
#df.set_index('index')
for i in range(100):

    # def timer(innerfun):
    #     tmp_t=time.time()
    #     tmp_r=innerfun()
    #     return time.time()-tmp_t,tmp_r

    # tmp_t=time.time()
    # env2=gym.make('jss-v1', env_config={'instance_path': 'resources/jsp_instances/standard/abz8.txt'})
    # t_1=(time.time()-tmp_t)
    # df['setuptime']['klagenfurt']+=t_1/100
    tmp_t=time.time()
    env_3=jss_lite(instance_path='resources/jsp_instances/standard/abz8.txt')
    t_1=time.time()-tmp_t
    df['setuptime']['lite']+=t_1/100




    # tmp_t=time.time()
    # path='resources/jsp_instances/standard/abz8.txt'
    # curr_instance=src.jsp_instance_parser.parse_jps_standard_specification(path)
    # res,std_matrix=curr_instance
    # env_alex = jss_env.DisjunctiveGraphJssEnv(res,default_visualisations='gantt_console')
    # t_2=(time.time()-tmp_t)
    # df['setuptime']['alex']+=t_2/100
    # tmp_t=time.time()
    # t=env2.reset()
    # t_1=(time.time()-tmp_t)
    # df['resettime']['klagenfurt']+=t_1/100
    # tmp_t=time.time()
    # t2=env_alex.reset()
    # t_2=(time.time()-tmp_t)
    # df['resettime']['alex']+=t_2/100
    tmp_t=time.time()
    t=env_3.reset()
    t_1=(time.time()-tmp_t)
    df['resettime']['lite']+=t_1/100


    # state=env2.reset()
    # for i in range(50):
    #     legal_action=state['action_mask']
    #     action=np.random.choice(len(legal_action), 1, p=(legal_action / legal_action.sum()))[0]
    #     tmp_t=time.time()
    #     state, reward, done, info=env2.step(action)
    #     t_1=(time.time()-tmp_t)
    #     df['50xactiontime']['klagenfurt']+=t_1/50

    #     if done==True:
    #         print(f"finished! no legal actions possible after {i} steps")
    #         break
    state=env_3.reset()
    for i in range(50):
        legal_action=env_3.get_legal_actions(state)
        action=np.random.choice(len(legal_action), 1, p=(legal_action / legal_action.sum()))[0]
        tmp_t=time.time()
        state, reward, done, info=env_3.step(action)
        t_1=(time.time()-tmp_t)
        df['50xactiontime']['lite']+=t_1/50

        if done==True:
            print(f"finished! no legal actions possible after {i} steps")
            break



    # action_list=np.arange(0,env_alex.n_jobs*env_alex.n_machines)
    # for i in range(50):
    #     action_mask=env_alex.valid_action_mask()
    #     actions=action_list[action_mask]
    #     next_action=random.choice(actions)
    #     #next_action=actions[action_mask[0]]
    #     #print(next_action)
    #     tmp_t=time.time()
    #     state,reward,done,info =env_alex.step(next_action)
    #     t_2=(time.time()-tmp_t)
    #     df['50xactiontime']['alex']+=t_2/50
    #     if done == True:
    #         print(f"finished after {i} steps")
    #         break
        #env.render()


print(df)



In [ ]:
import resource
import plotly.express as px
import pandas as pd
from datetime import datetime, timedelta
dt = datetime(2022, 1, 1, 0, 0, 0)
#stores quatuple(job,task,start_time,finish_time) 
time_now=time.time()
production_list=env_3.production_list
def production_to_dict(input,i,j):
    return(dict(Task=f"job_{input[0]}", Start=dt + timedelta(seconds=int(input[2])),Finish=dt + timedelta(seconds=int(input[3])),Resource=f"machine_{i}"))
df=pd.DataFrame()
liste=[]
for i in range(10):
    for j in range(10):
        if production_list[i,j] is not None:
            liste.append(production_to_dict(production_list[i,j],i,j))

df_2=pd.DataFrame(liste)
fig = px.timeline(df_2, x_start="Start", x_end="Finish", y="Task", color="Resource")
fig.show()

df = pd.DataFrame([
    dict(Task="Job A", Start='2009-01-01', Finish='2009-02-28', Resource="Alex"),
    dict(Task="Job B", Start='2009-03-05', Finish='2009-04-15', Resource="Alex"),
    dict(Task="Job C", Start='2009-02-20', Finish='2009-05-30', Resource="Max")
])

#fig = px.timeline(df, x_start="Start", x_end="Finish", y="Resource", color="Resource")
#fig.show()